In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns


In [12]:
from google.colab import files
data = files.upload()

Saving amazon_cells_labelled.txt to amazon_cells_labelled (1).txt


In [0]:
import io
amz = pd.read_csv(io.BytesIO(data['amazon_cells_labelled.txt']), delimiter= '\t', header=None)

In [14]:
amz.columns = ['message', 'positive']
amz.head()

,message,positive
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [15]:
#Version 2

keywords = ['great' , 'awesome' , 'love' , 'excellent' , 'good' , 'bonus' , 'nice' , 'very good' , 'very nice']

for key in keywords:

    pat = "(?i)(?:$|^| )({})(?:$|^| )".format(key)
    amz[str(key)] = amz.message.str.contains(pat = pat, regex=True)
# This is a really efficient loop. I don't know why it stood out 

amz['positive'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/usr

1    500
0    500
Name: positive, dtype: int64

In [16]:
amz.head(15)


# question here. number 1 has false for good but true for excellent. number 2 has false for great as does number 4
# number 10 says great but also reads false.

,message,positive,great,awesome,love,excellent,good,bonus,nice,very good,very nice
0,So there is no way for me to plug it in here i...,0,False,False,False,False,False,False,False,False,False
1,"Good case, Excellent value.",1,False,False,False,True,True,False,False,False,False
2,Great for the jawbone.,1,True,False,False,False,False,False,False,False,False
3,Tied to charger for conversations lasting more...,0,False,False,False,False,False,False,False,False,False
4,The mic is great.,1,False,False,False,False,False,False,False,False,False
5,I have to jiggle the plug to get it to line up...,0,False,False,False,False,False,False,False,False,False
6,If you have several dozen or several hundred c...,0,False,False,False,False,False,False,False,False,False
7,If you are Razr owner...you must have this!,1,False,False,False,False,False,False,False,False,False
8,"Needless to say, I wasted my money.",0,False,False,False,False,False,False,False,False,False
9,What a waste of money and time!.,0,False,False,False,False,False,False,False,False,False


In [0]:
data = amz[keywords]
target = amz['positive']

In [18]:
#Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 353


In [19]:
#-------------- Below here I am testing out things we worked on in the lesson-----------------------
from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

#--- I can identify bad reviews well but i do not identify good reviews well
# This goes in line with the observation above about odd classifications from the loop.

array([[485,  15],
       [338, 162]])

In [0]:
# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))



With 20% Holdout: 0.605
Testing on Sample: 0.647


In [0]:
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

# Here I am doing cross validation. which is the same process above but iterated n times
#consistent scores are good but my model is poor but still consistent. this is like
#t-tests again. 

array([0.68, 0.69, 0.72, 0.67, 0.64, 0.64, 0.62, 0.58, 0.65, 0.58])

In [0]:
amz['predictions']= y_pred
amz['correct'] = amz['positive'] == amz['predictions']
amz.head(25)

,message,positive,great,awesome,love,excellent,good,bonus,nice,predictions,correct
0,So there is no way for me to plug it in here i...,0,False,False,False,False,False,False,False,0,True
1,"Good case, Excellent value.",1,False,False,False,True,True,False,False,1,True
2,Great for the jawbone.,1,True,False,False,False,False,False,False,1,True
3,Tied to charger for conversations lasting more...,0,False,False,False,False,False,False,False,0,True
4,The mic is great.,1,False,False,False,False,False,False,False,0,False
5,I have to jiggle the plug to get it to line up...,0,False,False,False,False,False,False,False,0,True
6,If you have several dozen or several hundred c...,0,False,False,False,False,False,False,False,0,True
7,If you are Razr owner...you must have this!,1,False,False,False,False,False,False,False,0,False
8,"Needless to say, I wasted my money.",0,False,False,False,False,False,False,False,0,True
9,What a waste of money and time!.,0,False,False,False,False,False,False,False,0,True
